# Upper Green PEST Pilot Point Setup - Only APEX model

In [1]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu
import shapefile #the pyshp module
from pyemu.pst.pst_utils import SFMT,IFMT,FFMT,pst_config

flopy is installed in C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\flopy


# <span style="color:blue">1. Set up pilot points network for UG model some</span>.

There are multiple approaches to implementing pilot points with PEST++.  

In this class, we will use some kick-ass pyemu sweetness

### 1.1. Set up zones for where pilot points will be interpolated

We can have pilot point networks in multiple zones. In this case, we will make a simple zone file using `IBOUND` such that all active cells are in the same interpolation zone.

In [2]:
working_dir = 'D:/Projects/Watersheds/Green/Analysis/APEXs/TxtInOut'
os.chdir(working_dir)




# Build instruction files (Streamflow / Watertable)

### 1. Streamflow (output.rch)

In [3]:
from apexmf_pkgs import apexmf_pst_utils, apexmf_pst_par

# Create Template file

In [4]:
# Create parm template file

sw_par = apexmf_pst_utils.parm_to_tpl_file()
sw_par

,0,1,2,3,4,5,6,7,8,9
0,.50000,2.5000,1.0E-5,1.0000,~p5 ~,.20000,.00320,1.0E-5,1.0000,7.0000
1,.10000,~p12 ~,.20000,2.0000,~p15 ~,~p16 ~,~p17 ~,1.500,0.003,~p20 ~
2,10.00,0.90,~p23 ~,0.1000,~p25 ~,0.000,0.300,5.000,.01,1.100
3,0.30,1.00,2.5000,~p34 ~,0.9900,0.,0.200,1.000,.01,~p40 ~
4,0.50,~p42 ~,0.1500,~p44 ~,3.0000,0.500,1.000,0.000,~p49 ~,~p50 ~
5,~p51 ~,10.00,0.9000,0.6000,0.2000,0.900,1.000,0.600,0.100,7.000
6,~p61 ~,0.25,1.1000,1.0000,0.0010,3.000,10.000,0.500,0.500,1.
7,1.15,.05,0.5000,0.2000,0.8000,.05,0.000,10.000,1.000,.2
8,0.00,3.,0.5100,0.0001,0.0001,0.010,0.010,0.001,~p89 ~,~p90 ~
9,~p91 ~,~p92 ~,1.,1.,~p95 ~,1.,.9,1.,1.,.000548


# 1.2. Build instruction files (streamflow / watertable / baseflow)
## 1.2.1. Streamflow (*.rch)

In [5]:
# file path
rch_file = 'SITE199.RCH'
# reach numbers that are used for calibration
subs = [9, 96, 199]
# extract month_streamflow
apexmf_pst_utils.extract_month_str(rch_file, subs, '1/1/1990', '1/1/2000', '12/31/2012')

cha_009.txt file has been created...
cha_096.txt file has been created...
cha_199.txt file has been created...
Finished ...


### 1.2.3. Create instruction files for each str_sim file using the 'stf_mon.obd' file

In [6]:
# because we have 3 streamgages let's loop for them
# read streamobd and get column names
stf_obd = pd.read_csv(
                    'stf_mon.obd',
                    sep='\t',
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                    )
obds = stf_obd.columns.tolist()[::-1]
# obds.remove('sub046')
# obds.remove('sub130')
print(obds)
sim_files = ['cha_{:03d}.txt'.format(x) for x in subs]
# sed_files = ['sed_{:03d}.txt'.format(x) for x in subs]
# sim_files = sim_files + sed_files
print(sim_files)

['sub009', 'sub096', 'sub199']
['cha_009.txt', 'cha_096.txt', 'cha_199.txt']


In [7]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    apexmf_pst_utils.stf_obd_to_ins(sim_files[i], obds[i], '1/1/2000', '12/31/2012', time_step='month')

cha_009.txt.ins file has been created...
cha_096.txt.ins file has been created...
cha_199.txt.ins file has been created...


In [8]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'ug_dummy.pst')

# print(os.chdir(".."))
io_files

C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\pyemu\pst\pst_utils.py:615: UserWarning: error processing instruction file cha_009.txt.ins, trying inschek: substring not found
  warnings.warn("error processing instruction file {0}, trying inschek: {1}".format(ins_file,str(e)))
C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\pyemu\pst\pst_utils.py:615: UserWarning: error processing instruction file cha_096.txt.ins, trying inschek: substring not found
  warnings.warn("error processing instruction file {0}, trying inschek: {1}".format(ins_file,str(e)))
C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\pyemu\pst\pst_utils.py:615: UserWarning: error processing instruction file cha_199.txt.ins, trying inschek: substring not found
  warnings.warn("error processing instruction file {0}, trying inschek: {1}".format(ins_file,str(e)))
error using inschek for instruction file cha_009.txt.ins:run() returned non-zero: 1
observations in this instructio

(['PARM1501.DAT.tpl'],
 ['PARM1501.DAT'],
 ['cha_009.txt.ins', 'cha_096.txt.ins', 'cha_199.txt.ins'],
 ['cha_009.txt', 'cha_096.txt', 'cha_199.txt'])

The ``parse_dir_for_io_files()`` helper is looking for files with the ".tpl" and ".ins" extension.  This assumes that the corresponding model input and model output files are the same name, minus the ".tpl" and ".ins" extension, respectively.  These file lists are then passed to another helper, which builds a basic control file for you (``Pst.from_io_files()``).  Let's look at this generic ``Pst`` instance:

In [9]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
p12,p12,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p15,p15,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p16,p16,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p17,p17,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p20,p20,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p23,p23,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p25,p25,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p34,p34,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p40,p40,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p42,p42,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1


In [10]:
par = par.sort_values(by=['pargp', 'parnme'])
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
p12,p12,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p15,p15,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p16,p16,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p17,p17,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p20,p20,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p23,p23,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p25,p25,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p34,p34,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p40,p40,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p42,p42,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1


# APEX

In [11]:
pst.parameter_data = apexmf_pst_utils.export_pardb_pest(par)

In [12]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,absolute_lower,absolute_upper,default_initial,flag
name,,,,,,,,,,,,,,
p12,p12,log,factor,2.50000,1.50000,2.5000,pargp,1.0,0.0,1,1.5000,2.5000,2.5000,y
p15,p15,log,factor,0.00001,0.00001,0.3000,pargp,1.0,0.0,1,0.0000,0.3000,0.0000,y
p16,p16,log,factor,1.00000,1.00000,1.5000,pargp,1.0,0.0,1,1.0000,1.5000,1.0000,y
p17,p17,log,factor,0.25000,0.00001,0.5000,pargp,1.0,0.0,1,0.0000,0.5000,0.2500,y
p20,p20,log,factor,0.20000,0.05000,0.4000,pargp,1.0,0.0,1,0.0500,0.4000,0.2000,y
p23,p23,log,factor,0.00320,0.00230,0.0032,pargp,1.0,0.0,1,0.0023,0.0032,0.0032,y
p25,p25,log,factor,0.00001,0.00001,2.0000,pargp,1.0,0.0,1,0.0000,2.0000,0.0000,y
p34,p34,log,factor,0.50000,0.50000,0.6000,pargp,1.0,0.0,1,0.5000,0.6000,0.5000,y
p40,p40,log,factor,1.00000,0.00100,1.0000,pargp,1.0,0.0,1,0.0010,1.0000,1.0000,y



Cool - the other tpl files were found and parsed - parameter listed in them were added to the control file.  But we have generic entries for initial values bounds...

## Observation

In [13]:
obd = pst.observation_data
print(obd)

                      obsnme        obsval  weight  obgnme
sub009_200001  sub009_200001  1.000000e+10     1.0  obgnme
sub009_200002  sub009_200002  1.000000e+10     1.0  obgnme
sub009_200003  sub009_200003  1.000000e+10     1.0  obgnme
sub009_200004  sub009_200004  1.000000e+10     1.0  obgnme
sub009_200005  sub009_200005  1.000000e+10     1.0  obgnme
...                      ...           ...     ...     ...
sub199_201208  sub199_201208  1.000000e+10     1.0  obgnme
sub199_201209  sub199_201209  1.000000e+10     1.0  obgnme
sub199_201210  sub199_201210  1.000000e+10     1.0  obgnme
sub199_201211  sub199_201211  1.000000e+10     1.0  obgnme
sub199_201212  sub199_201212  1.000000e+10     1.0  obgnme

[468 rows x 4 columns]


In [14]:
for i in range(len(obd)):
    if obd.iloc[i, 0][:6] == 'sub009':
        obd.iloc[i, 3] = 'sub009'
    elif obd.iloc[i, 0][:6] == 'sub096':
        obd.iloc[i, 3] = 'sub096'
    elif obd.iloc[i, 0][:6] == 'sub199':
        obd.iloc[i, 3] = 'sub199'
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:7]


In [15]:
print(obd)

                      obsnme        obsval  weight  obgnme
sub009_200001  sub009_200001  1.000000e+10     1.0  sub009
sub009_200002  sub009_200002  1.000000e+10     1.0  sub009
sub009_200003  sub009_200003  1.000000e+10     1.0  sub009
sub009_200004  sub009_200004  1.000000e+10     1.0  sub009
sub009_200005  sub009_200005  1.000000e+10     1.0  sub009
...                      ...           ...     ...     ...
sub199_201208  sub199_201208  1.000000e+10     1.0  sub199
sub199_201209  sub199_201209  1.000000e+10     1.0  sub199
sub199_201210  sub199_201210  1.000000e+10     1.0  sub199
sub199_201211  sub199_201211  1.000000e+10     1.0  sub199
sub199_201212  sub199_201212  1.000000e+10     1.0  sub199

[468 rows x 4 columns]


## 2.3. Import measured data

In [16]:
stf_obd = pd.read_csv('stf_mon.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd['1/1/2000': '12/31/2012']
# stf_obd = stf_obd.drop(['sub046', 'sub130'], axis=1)
stf_obd =  stf_obd.reindex(sorted(stf_obd.columns), axis=1)
stf_obd

,sub009,sub096,sub199
date,,,
2000-01-31,3.87,10.64,31.30
2000-02-29,3.69,10.42,31.41
2000-03-31,4.28,17.23,34.44
2000-04-30,12.08,38.66,36.22
2000-05-31,29.29,70.10,39.65
...,...,...,...
2012-08-31,8.11,17.41,29.86
2012-09-30,4.49,11.31,25.16
2012-10-31,3.14,13.30,24.11


In [17]:
# Get sub list based on obd order
sub_order = []
for i in obd.obgnme.tolist():
    if i not in sub_order:
        sub_order.append(i)
sub_order


['sub009', 'sub096', 'sub199']

In [18]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in sub_order[:3]:
    tot_obd += stf_obd[i].dropna().tolist()
for j in sub_order[3:]:
    tot_obd += gwt_obd[j].dropna().tolist()    
len(tot_obd)
# tot_obd

468

In [19]:
obd.loc[:, 'obsval'] = tot_obd
obd

,obsnme,obsval,weight,obgnme
sub009_200001,sub009_200001,3.87,1.0,sub009
sub009_200002,sub009_200002,3.69,1.0,sub009
sub009_200003,sub009_200003,4.28,1.0,sub009
sub009_200004,sub009_200004,12.08,1.0,sub009
sub009_200005,sub009_200005,29.29,1.0,sub009
...,...,...,...,...
sub199_201208,sub199_201208,29.86,1.0,sub199
sub199_201209,sub199_201209,25.16,1.0,sub199
sub199_201210,sub199_201210,24.11,1.0,sub199
sub199_201211,sub199_201211,24.55,1.0,sub199


### 4. Export control file

In [20]:
pst.control_data.noptmax=0
pst.model_command = 'python forward_run.py'
pst.write('green_pest.pst')

noptmax:0, npar_adj:21, nnz_obs:468


also cool - the instruction files in the directory were also found and parsed so that observation listed in the instruction files were added as well. There are some subtlies here, but we will skip them for now.

## Regularization

Regularization is ....

in pyemu, we can add two forms of regularization:
- preferred value: we want the parameter values to stay as close to the initial values as possible
- preferred difference: we prefer the differences in parameter values to be minimized

Preferred value is easy to understand, we simply add ``prior_information`` to the control file to enforce this condition.  pyemu uses a helper for this:

In [36]:
# load the pre-constructed pst
pst = pyemu.Pst(os.path.join(working_dir,pst_name))

In [37]:
pyemu.helpers.zero_order_tikhonov(pst,parbounds=False)

In [38]:
pst.prior_information

,pilbl,equation,obgnme,weight
0,hk00,1.0 * log(hk00) = 6.989700E-01,regulhk,1.0
1,hk01,1.0 * log(hk01) = 6.989700E-01,regulhk,1.0
2,hk02,1.0 * log(hk02) = 6.989700E-01,regulhk,1.0
3,hk03,1.0 * log(hk03) = 6.989700E-01,regulhk,1.0
4,hk04,1.0 * log(hk04) = 6.989700E-01,regulhk,1.0
5,hk05,1.0 * log(hk05) = 6.989700E-01,regulhk,1.0
6,hk06,1.0 * log(hk06) = 6.989700E-01,regulhk,1.0
7,hk07,1.0 * log(hk07) = 6.989700E-01,regulhk,1.0
8,hk08,1.0 * log(hk08) = 6.989700E-01,regulhk,1.0
9,hk09,1.0 * log(hk09) = 6.989700E-01,regulhk,1.0


Ok, that's fine, but should the weight on preferring HK not to change be the same as preferring recharge not to change?  Seems like we would want recharge to change less than HK.  This preference can be expressed by using the parameter bounds to form the weights

In [43]:
pyemu.helpers.zero_order_tikhonov(pst,parbounds=True)

In [44]:
pst.prior_information

,pilbl,equation,obgnme,weight
pilbl,,,,
hk00,hk00,1.0 * log(hk00) = 6.989700E-01,regulhk,0.500000
hk01,hk01,1.0 * log(hk01) = 6.989700E-01,regulhk,0.500000
hk02,hk02,1.0 * log(hk02) = 6.989700E-01,regulhk,0.500000
hk03,hk03,1.0 * log(hk03) = 6.989700E-01,regulhk,0.500000
hk04,hk04,1.0 * log(hk04) = 6.989700E-01,regulhk,0.500000
hk05,hk05,1.0 * log(hk05) = 6.989700E-01,regulhk,0.500000
hk06,hk06,1.0 * log(hk06) = 6.989700E-01,regulhk,0.500000
hk07,hk07,1.0 * log(hk07) = 6.989700E-01,regulhk,0.500000
hk08,hk08,1.0 * log(hk08) = 6.989700E-01,regulhk,0.500000


Now we are really preferring recharge not to change...good!

So what about preferred difference regularization?  Well pyemu can do that too.  Remember that ``Cov``ariance matrix we built above? It expresses the spatial relationship between pilot points, so we use to setup these prior information equations:

In [45]:
pyemu.helpers.first_order_pearson_tikhonov(pst,cov)

getting CC matrix


Exception: Matrix.indices(): name not found: porosity

In [46]:
pst.prior_information

,pilbl,equation,obgnme,weight
pilbl,,,,
hk00,hk00,1.0 * log(hk00) = 6.989700E-01,regulhk,0.500000
hk01,hk01,1.0 * log(hk01) = 6.989700E-01,regulhk,0.500000
hk02,hk02,1.0 * log(hk02) = 6.989700E-01,regulhk,0.500000
hk03,hk03,1.0 * log(hk03) = 6.989700E-01,regulhk,0.500000
hk04,hk04,1.0 * log(hk04) = 6.989700E-01,regulhk,0.500000
hk05,hk05,1.0 * log(hk05) = 6.989700E-01,regulhk,0.500000
hk06,hk06,1.0 * log(hk06) = 6.989700E-01,regulhk,0.500000
hk07,hk07,1.0 * log(hk07) = 6.989700E-01,regulhk,0.500000
hk08,hk08,1.0 * log(hk08) = 6.989700E-01,regulhk,0.500000


What happened?  We replace the preferred value equations with a bunch of new equations.  These equations each include two parameter names and have different weights - can you guess what the weights are?  The weights are the pearson correlation coefficients (CC) between the pilot points (remember those from way back?).  These CC values are calculated from the covariance matrix, which is implied by the geostatistical structure...whew! 

# For river Bed conductance interpolation

In [62]:
# interpolate the pilot point values to the grid
riv_cond = pyemu.gw_utils.fac2real(ppf,factors_file=ppf+".fac",out_file=None)

In [64]:
np.shape(riv_cond)

(154, 268)

## 1. get only river grids

In [139]:
df_riv = pd.read_csv(
                    shpwd + "\\mf\\ss_072519.riv",
                    delim_whitespace=True,
                    skiprows=3,
#                     usecols=[1,2],
                    header=None
                    )

In [136]:
new_riv_cf = [riv_cond[df_riv.iloc[i, 0], df_riv.iloc[i, 1]] for i in range(len(df_riv))]

In [137]:
df_riv.iloc[:, 4] = new_riv_cf
df_riv.iloc[:, 4] = df_riv.iloc[:, 4].map(lambda x: '{:.10e}'.format(x))
df_riv.iloc[:, 3] = df_riv.iloc[:, 3].map(lambda x: '{:.10e}'.format(x))
df_riv.iloc[:, 5] = df_riv.iloc[:, 5].map(lambda x: '{:.10e}'.format(x))

In [138]:
with open(os.path.join(shpwd + "\\mf", "ss_072519.riv"), 'w') as f:
    f.write("# RIV: River package file created on 7/25/2019 by ModelMuse version 4.0.0.0." + "\n")
    f.write("  1467     9 AUXILIARY IFACE # DataSet 2: MXACTC IRIVCB Option" + "\n")
    f.write("  1467     0 # Data Set 5: ITMP NP Stress period 1" + "\n")    
    df_riv.to_csv(f, sep='\t',
                  header=False,
                  index=False,
#                   float_format='%.2f', 
                  line_terminator='\n', 
                  encoding='utf-8')


# Build instruction files (Streamflow / Watertable)

### 1. Streamflow (channel_day.txt)

In [68]:
import csv

wd2 = "D:\\Projects\\MiddleBosque\\Analysis\\SWAT+MODFLOW Model_middle_bosque"
df_str = pd.read_csv(
                    wd2 + "\\channel_day - Copy.txt",
                    delim_whitespace=True,
                    skiprows=3,
#                     usecols=[1,2],
                    header=None
                    )
test = []
for i in range(len(df_str)):
    if df_str.iloc[i, 6] == 'cha53':
        a = 'l1 w w w w w w w w w '
        b = '!str_{}{:02d}{:02d}!'.format(df_str.iloc[i, 3], df_str.iloc[i, 1], df_str.iloc[i, 2])
        test.append(a+b)
    else:
        a = 'l1'
        test.append(a)


with open('str.ins', "w", newline='') as f:
    f.write("pif ~" + "\n")  
    writer = csv.writer(f)
    for row in test:
        writer.writerow([row])


### 2. Watertable (modflow_cell_obs.txt)

In [49]:
import csv
import pandas as pd
import numpy as np

st_date = '1/1/1980'
wd2 = "D:\\Projects\\MiddleBosque\\Analysis\\SWAT+MODFLOW Model_middle_bosque"
df_wt = pd.read_csv(
                    wd2 + "\\modflow_cell_obs.txt",
                    delim_whitespace=True,
                    skiprows=1,
#                     usecols=[1,2],
                    header=None
                    )
df_wt.index = pd.date_range(st_date, periods=len(df_wt))
# print(df_str)

co1L = []
co2D = []



# for i in range(len(df_str)):
#     if ((df_wt.index[i].strftime('%Y%m%d') >= '19850821') & (df_wt.index[i].strftime('%Y%m%d') <= '19860507')):
#         print('true')
df_wt['date'] = df_wt.index.strftime('%Y%m%d')
# df_wt['test'] = CO1L['8/21/1985':'5/7/1986']
# df_wt['test'] = df_wt[(df_wt.index >= '08/21/1985') & (df_wt.index <= '05/07/1986')] = 'w'
df_wt['2nd'] = np.where((df_wt.index >= '08/21/1985') & (df_wt.index <= '05/07/1986'), ('l1 !wt_2nd'+df_wt['date']+'!'), 'l1 w')
df_wt['1st'] = np.where((df_wt.index >= '09/30/1985') & (df_wt.index <= '04/01/1986'), ('!wt_1nd'+df_wt['date']+'!'), ' w')


print(df_wt)

with open('wt.ins', "w", newline='') as f:
    f.write("pif ~" + "\n")
    df_wt.to_csv(f, sep='\t',
                  header=False,
                  index=False,
#                   float_format='%.2f', 
                  line_terminator='\n', 
                  columns=('2nd','1st'),
                  encoding='utf-8')

'''
test = []
for i in range(len(df_str)):
    if df_str.iloc[i, 6] == 'cha53':
        a = 'l1 w w w w w w w w '
        b = '!str_{}{:02d}{:02d}!'.format(df_str.iloc[i, 3], df_str.iloc[i, 1], df_str.iloc[i, 2])
        test.append(a+b)
    else:
        a = 'l1'
        test.append(a)


with open('str.ins', "w", newline='') as f:
    f.write("pif ~" + "\n")  
    writer = csv.writer(f)
    for row in test:
        writer.writerow([row])
'''

                   0         1      date   2nd 1st
1980-01-01  255.5776  253.4422  19800101  l1 w   w
1980-01-02  255.6870  253.5240  19800102  l1 w   w
1980-01-03  255.7828  253.5971  19800103  l1 w   w
1980-01-04  255.8695  253.6645  19800104  l1 w   w
1980-01-05  255.9489  253.7285  19800105  l1 w   w
...              ...       ...       ...   ...  ..
2005-12-27  263.4627  261.2057  20051227  l1 w   w
2005-12-28  263.4421  261.1927  20051228  l1 w   w
2005-12-29  263.4519  261.1992  20051229  l1 w   w
2005-12-30  263.4284  261.1831  20051230  l1 w   w
2005-12-31  263.4290  261.1830  20051231  l1 w   w

[9497 rows x 5 columns]


'\ntest = []\nfor i in range(len(df_str)):\n    if df_str.iloc[i, 6] == \'cha53\':\n        a = \'l1 w w w w w w w w \'\n        b = \'!str_{}{:02d}{:02d}!\'.format(df_str.iloc[i, 3], df_str.iloc[i, 1], df_str.iloc[i, 2])\n        test.append(a+b)\n    else:\n        a = \'l1\'\n        test.append(a)\n\n\nwith open(\'str.ins\', "w", newline=\'\') as f:\n    f.write("pif ~" + "\n")  \n    writer = csv.writer(f)\n    for row in test:\n        writer.writerow([row])\n'

In [148]:
a = np.full((33, 55), 10)

In [149]:
np.shape(a)

(33, 55)

In [155]:
np.savetxt('test.txtt', a, fmt='%.12e', delimiter='\t')
np.savetxt('vtest.txtt', a/10, fmt='%.12e', delimiter='\t')

In [154]:
b = np.loadtxt('test.txtt')
b/10

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [111]:
with open('hk04.dat', 'r') as f:
    data = [x.strip().split() for x in f if x.strip()]
hk = float(data[0][4])
hk

1.0

In [120]:
data_fac = ['hk01.dat', 'hk02.dat', 'hk03.dat', 'sy01.dat', 'sy02.dat', 'sy03.dat']
for i in data_fac:
    if i[:2] == 'hk':
        print('true')



true
true
true


In [118]:
print(a)

hk01.dat.ref


In [119]:
b/10

TypeError: unsupported operand type(s) for /: 'str' and 'int'